In [1]:
!pip install facenet-pytorch
!pip install torch torchvision


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 75.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166

In [2]:
!pip uninstall pillow -y
!pip install pillow


Found existing installation: pillow 10.2.0
Uninstalling pillow-10.2.0:
  Successfully uninstalled pillow-10.2.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 34.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
facenet-pytorch 2.6.0 requires Pillow<10.3.0,>=10.2.0, but you have pillow 11.1.0 which is incompatible.
mlxtend 0.23.3 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
plotnine 0.14.4 requires matplotlib>=3.8.0, but you have matplotlib 3.7.5 which is incompatible.


In [3]:
import os
import numpy as np
from PIL import Image
from facenet_pytorch import MTCNN
import torch

# Configuration
INPUT_DIR = "/kaggle/input/affectnet-cleaned"  # Input dataset root (contains Train and Test folders)
OUTPUT_DIR = "/kaggle/working/cropped_dataset"  # Directory to save cropped images
IMG_HEIGHT, IMG_WIDTH = 331, 331

# Automatically determine device
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Initialize MTCNN for face detection
mtcnn = MTCNN(keep_all=False, device=device)  # Set keep_all=False to focus on single largest face

# Step 1: Preprocess Image (Resize Before MTCNN)
def preprocess_image(image_path, target_size=(512, 512)):
    """
    Preprocess the image by resizing it to a fixed size.

    Args:
        image_path (str): Path to the image.
        target_size (tuple): Target size to resize the image.

    Returns:
        PIL.Image: Resized image.
    """
    img = Image.open(image_path).convert("RGB")
    img = img.resize(target_size)
    return img

# Step 2: Crop and Verify Face
def crop_and_verify_face(image_path):
    """
    Crop the largest detected face from the image and verify it.

    Args:
        image_path (str): Path to the image.

    Returns:
        PIL.Image or None: Cropped and verified face image if valid, otherwise None.
    """
    img = preprocess_image(image_path)
    img_array = np.array(img)
    img_height, img_width, _ = img_array.shape
    boxes, _ = mtcnn.detect(img_array)

    if boxes is not None:
        # Find the largest face
        areas = [(x2 - x1) * (y2 - y1) for x1, y1, x2, y2 in boxes]
        largest_box = boxes[np.argmax(areas)].astype(int)

        # Ensure bounding box values are within image boundaries
        x1 = max(0, largest_box[0])
        y1 = max(0, largest_box[1])
        x2 = min(img_width, largest_box[2])
        y2 = min(img_height, largest_box[3])

        # Validate bounding box dimensions
        if x2 > x1 and y2 > y1:  # Ensure positive width and height
            cropped = img_array[y1:y2, x1:x2]

            if cropped.size > 0:  # Ensure the crop is valid
                # Resize cropped image and verify again
                cropped_img = Image.fromarray(cropped).resize((IMG_WIDTH, IMG_HEIGHT))
                cropped_img_array = np.array(cropped_img)
                boxes_after_crop, _ = mtcnn.detect(cropped_img_array)

                if boxes_after_crop is not None:  # Face exists after cropping
                    return cropped_img
                else:
                    print(f"No face detected after cropping in {image_path}")
            else:
                print(f"Invalid crop size for {image_path}: {cropped.shape}")
        else:
            print(f"Invalid bounding box for {image_path}: {largest_box}")
    else:
        print(f"No face detected in {image_path}")

    return None  # Return None if no valid crop is possible

# Step 3: Crop All Images
def crop_all_images(dataset_dir, output_dir):
    """
    Crop and verify faces from all images in the dataset and save the results.

    Args:
        dataset_dir (str): Path to the dataset directory.
        output_dir (str): Path to save cropped images.
    """
    # Iterate through Train and Test splits
    for split in ["Train", "Test"]:
        split_dir = os.path.join(dataset_dir, split)
        output_split_dir = os.path.join(output_dir, split)

        if not os.path.exists(split_dir):
            print(f"Directory '{split_dir}' does not exist. Skipping...")
            continue

        # Process each class directory
        class_dirs = [os.path.join(split_dir, d) for d in os.listdir(split_dir) if os.path.isdir(os.path.join(split_dir, d))]
        for class_dir in class_dirs:
            class_name = os.path.basename(class_dir)
            output_class_dir = os.path.join(output_split_dir, class_name)
            os.makedirs(output_class_dir, exist_ok=True)

            images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
            total_images = len(images)
            print(f"Processing images for class '{class_name}' ({total_images} images)...")

            for i, img_name in enumerate(images, start=1):
                img_path = os.path.join(class_dir, img_name)
                cropped_face = crop_and_verify_face(img_path)

                if cropped_face is not None:
                    cropped_path = os.path.join(output_class_dir, img_name)  # Keep the original filename
                    cropped_face.save(cropped_path)
                    print(f"[{i}/{total_images}] Cropped, verified, and saved: {img_name}")
                else:
                    print(f"[{i}/{total_images}] Skipped (no valid face detected): {img_name}")

            print(f"Finished processing for class '{class_name}'. {total_images} images processed.")

# Step 4: Main Function
def main():
    crop_all_images(INPUT_DIR, OUTPUT_DIR)
    print("Cropping completed. All results saved.")

if __name__ == "__main__":
    main()


Using device: cpu
Directory '/kaggle/input/affectnet-cleaned/Train' does not exist. Skipping...
Directory '/kaggle/input/affectnet-cleaned/Test' does not exist. Skipping...
Cropping completed. All results saved.
